In [ ]:
import librosa
import librosa.display
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import random
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import soundfile as sf
import random
import os, sys
import time
#import subprocess
#subprocess.run("pip install pydub", shell = True)
import pydub
from pydub import AudioSegment

In [ ]:
#適宜変える
#mainpath = '/content/drive/MyDrive/procon2022/'
mainpath = './'

In [ ]:
# 問題作成ツール
# 無音wavファイル: http://hitotose-switch.blogspot.com/2013/12/wav.html
class Question:
    SR = 48000 # サンプリング周波数
    
    # サンプリング値を秒に変換
    # こちらはサンプリング値でずらしたいが、Pydubは秒を欲しがる
    def rate2sec(rate):
        return rate / Question.SR
    
    # カット 頭と末尾のデータを刈り取る
    # 頭と末尾何秒「切り取るか?」を指定
    def cut_wave(x, head=0, tail=0):
        x = x[head:len(x)-tail]
        return x
    
    
    # 2つの波形を合成する(ループで呼び出して複数まぜる、全ての波形を合成)
    # カットをしてからここを呼び出す
    def mix_wave(mix, x, skip=0):
        mix = mix + np.pad(x, [skip, len(mix) - len(x) - skip])
        return mix
    
    # 32個の問題データを作る
    # "wave_file/"ディレクトリに作成します。ディレクトリを作っておいてください。
    # "muon_10.sec.wav"はカレントディレクトリに
    # サンプルデータは"JKspeech/"から引っ張ります
    def create(sellect): # sellect: E01、E02にするように 
        for lp in range(0, 32):
            #print(lp)
            #print(sellect)
            # 無音データ(重ね合わせ用)
            muon = AudioSegment.from_file(mainpath + "muon_10sec.wav", format="wav")
            muon = muon + muon + muon; #30秒の無音データ
            # 作成
            speech = np.empty(0) # 読みデータ
            offset = np.empty(0) # 開始位置
            mix = np.array(muon.get_array_of_samples())
            max_time = 0
            num = random.randint(2, 20)
            remain = np.empty(0)
            ej = ["E", "J"]
            for i in range(1, 45):
                remain = np.append(remain, str(i).zfill(2))
            
            #print("合成数: " + str(num))
            for i in range(0, num):
                r_mus = random.randint(0, remain.size - 1)
                r_ej = random.randint(0, 1)

                # はじめ半分はsellectを含む
                if i == num-1 and sellect != "" and lp < 16:
                    r_ej = 0 if sellect[0] == "E" else 1
                    r_mus = int(sellect[1:]) - 1
                
                
                # うしろ半分はsellectを含まない
                if lp >= 16 and remain[r_mus] == sellect[1:]:
                    r_ej = 1 if sellect[0] == "E" else 0 # sellectにない言語にする
                
                
                #path = "/content/drive/MyDrive/procon2022/JKspeech/" + ej[r_ej] + remain[r_mus] + ".wav" #勝手に変えました(content/drive)
                path = mainpath + "JKspeech/" + ej[r_ej] + remain[r_mus] + ".wav"
                speech = np.append(speech, ej[r_ej] + remain[r_mus])
                #print(path)
                base_sound_AS = AudioSegment.from_file(path, format="wav")
                base_sound = np.array(base_sound_AS.get_array_of_samples())
                #print("初期状態:" + str(base_sound.duration_seconds) + "秒") # カット前の秒数
                head_cut = random.randint(0, int(len(base_sound) - 48000)) # 1s以上確実に残して頭からカット
                tail_cut = random.randint(0, int(len(base_sound) - head_cut - 48000)) # 1s以上確実に残して後ろからカット
                base_sound = Question.cut_wave(base_sound, head = head_cut, tail = tail_cut)
                #print("最終状態:" + str(base_sound.duration_seconds) + "秒") # カット後の秒数
                skip_time = random.randint(0, max_time) #飛ばす時間
                offset = np.append(offset, skip_time)
                #print("スキップ時間:" + str(Question.rate2sec(skip_time)) + "秒")
                #max_time = max(max_time, skip_time + int(base_sound.duration_seconds * Question.SR))
                max_time = min(max(max_time, skip_time + len(base_sound)), 432000)
                #print("合計時間:" + str(Question.rate2sec(max_time)) + "秒/48000")
                #mix = Question.mix_wave(mix, base_sound, skip=skip_time) # 周波数単位でずらす
                mix = Question.mix_wave(mix, base_sound, skip=skip_time) # 周波数単位でずらす
                remain = np.delete(remain, r_mus)
            
            outfile_name = mainpath + "wave_file/information" + str(lp) + ".txt"
            f = open(outfile_name, mode='w')
            f.write("nspeech: " + str(num) + "\n")
            
            s = "speech: "
            for i in range(0, num):
                s += str(speech[i])
                if (i != num - 1):
                    s += ", "
            f.write(s)
            f.write("\n")
            
            s = "offset: "
            for i in range(0, num):
                s += str(int(offset[i]))
                if (i != num - 1):
                    s += ", "
            f.write(s)
            f.write("\n")
            
            f.close()
            
            #print("最終時間" + str(Question.rate2sec(max_time)) + "秒")
            #mix = mix[:int(Question.rate2sec(max_time) * 1000)]
            mix = mix[:min(max_time, 480000)]
            outfile_name = mainpath + "wave_file/mix" + str(lp) + ".wav"
            mix = AudioSegment(
                    mix.astype("int16").tobytes(), 
                    sample_width=base_sound_AS.sample_width, 
                    frame_rate=base_sound_AS.frame_rate, 
                    channels=base_sound_AS.channels,
                    )
            mix.export(outfile_name, format="wav")


In [ ]:
model = torchvision.models.resnet50()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model = model.to(device)

In [ ]:
"""
Target = [] # E01からJ44を作成
EJ = ["E", "J"]
for ej in EJ:
    for i in range(1, 45):
        Target.append(ej + str(i).zfill(2))

for target in Target:
    #GPUで保存
    if torch.cuda.is_available():
        model_path = mainpath + 'model/model_cuda_' + target + '.pth'
        torch.save(model.state_dict(), model_path)
    #CPUで保存
    else:
        model_path = mainpath + 'model/model_cpu_' + target + '.pth'
        torch.save(model.state_dict(), model_path)
"""

In [ ]:
#学習に使うメソッド
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
import time
from IPython.display import clear_output

In [ ]:
Target = [] # E01からJ44を作成
EJ = ["E", "J"]
for ej in EJ:
    for i in range(1, 45):
        Target.append(ej + str(i).zfill(2))


#学習
for learntimes in range(0, 500):
    print()
    print(learntimes)
    # 問題作成
    print("make problems...")
    question = Question()
    question.create()
    print("finish")

    #波形データを得る
    file_path = mainpath + "wave_file/"
    wave_name = np.array(["mix" + str(i) + ".wav" for i in range(0, 32)])
    #波形の長さが異なるのでどうにかしないといけない
    #そのままメルスペクトログラム化する
    #メルスペクトログラムの大きさを調節する
    X = np.empty(0)
    for name in wave_name:
        y, sr = librosa.load(mainpath + file_path + name)
        hop_length = len(y) // 224
        spec_log = librosa.power_to_db(np.abs(librosa.stft(y, n_fft=447, hop_length=hop_length)))
        X = np.append(X, spec_log[:224,:224])
    X = np.reshape(X, (-1,224,224))
    #print(X.shape)

    for target in Target:
        print(target, end = ' ')
        #モデルの読み込み
        #モデルを一度保存してないとエラーします（とりあえずはここのコード無視してね）
        #GPU
        if torch.cuda.is_available():
            model_path = mainpath + 'model/cnn_cuda_2d' + target + '.pth'
            model.load_state_dict(torch.load(model_path))
        #CPU
        else:
            model_path = mainpath + 'model/cnn_cpu_2d' + target + '.pth'
            model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

        #狙いの音声が使われているかを取る
        file_path = mainpath + "wave_file/"
        label_name = np.array(["information" + str(i) + ".txt" for i in range(0,32)])

        Y = np.empty(0)
        for name in label_name:
            f = open(file_path + name, 'r')
            s = f.read()
            #答えは[無い率,有る率]とする
            if target in s:
                Y = np.append(Y, 1)
            else:
                Y = np.append(Y, 0)

        #データセット作成
        #参考：https://dreamer-uma.com/pytorch-dataset/
        tensorX = torch.tensor(X, dtype=torch.float32)
        tensorY = torch.tensor(Y, dtype=torch.int64)
        Dataset = torch.utils.data.TensorDataset(tensorX, tensorY)

        #データローダー作成
        trainloader = torch.utils.data.DataLoader(dataset=Dataset,
                                                batch_size = 32,
                                                shuffle = True,
                                                num_workers = 0)

        #学習
        #参考 https://qiita.com/mathlive/items/8e1f9a8467fff8dfd03c
        train_loss_value = []
        train_acc_value = []

        BATCH_SIZE = 32
        EPOCH = 1
        for epoch in range(EPOCH):
            print("epoch:" + str(epoch))
            sum_loss = 0.0
            sum_correct = 0
            sum_total = 0
            for (inputs, labels) in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                #inputsの大きさが24000(0.5秒)じゃないとダメ
                inputs = inputs.unsqueeze(1)
                #print(inputs.shape)
                outputs = model(inputs)
                #結果出力
                sof = nn.Softmax(dim=1)(outputs)
                print(sof[0])
                print(labels[0])
                #
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                sum_loss += loss.item()

            clear_output(True)
            mean_loss = sum_loss*BATCH_SIZE/len(trainloader.dataset)
            print("train mean loss={}".format(mean_loss))
            train_loss_value.append(mean_loss)

        time.sleep(0.1)
        #GPUで保存
        if torch.cuda.is_available():
            model_path = mainpath + 'model/cnn_cuda_2d' + target + '.pth'
            torch.save(model.state_dict(), model_path)
        #CPUで保存
        else:
            model_path = mainpath + 'model/cnn_cpu_2d' + target + '.pth'
            torch.save(model.state_dict(), model_path)
